In [ ]:
from yadeGrid.body import Body, Quaternion
from yadeGrid.vectorFunc import norm, normalise, dotProduct, crossProduct
from yadeGrid.yadeTypes import Vector3D, F64
from yadeGrid.interaction import Interaction
from yadeGrid.simulations import ForceResetter, SerialEngine, Scene, SimulationLoop, InteractionsCalculator, LeapFrogIntegrator, BodyContainer, InteractionContainer, CustomPythonEngine
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.pyplot as plt 

In [ ]:


young: F64   = F64(70e9)
poisson: F64 = F64(0.35)
density: F64 = F64(2700.0)


bodyIds: list = [0, 1]
bodyPos: list[Vector3D] = [np.array([i, 0, 0]) for i in bodyIds]
radius: F64 = F64(0.1)
bodyCont = BodyContainer()
[bodyCont.add_body(pos=pos, radius=radius, density=density) for pos in bodyPos]


interCont = InteractionContainer()

rad     = F64(0.1)
density = F64(2700.0)
pos1    = np.array([0, 0, 0], dtype=F64)
pos2    = np.array([1, 0, 0], dtype=F64)
pos3    = np.array([3, 0, 0], dtype=F64)
young   = F64(70e9)
poisson = F64(0.35)

dt: F64 = F64(1e-6)

edges = [[0, 1]]
for edge in edges:
    b1 = bodyCont[edge[0]]
    b2 = bodyCont[edge[1]]
    interCont.add_interaction(b1, b2, dt, young_mod=young, poisson=poisson)

bodyCont[0].vel = np.array([0, 100, 0], dtype=F64)
bodyCont[0].DynamicQ = False

print(bodyCont.bodies[0].mass)
omega = Scene()


bodyPoses: list[list[Vector3D]] = []
bodyOris: list[list[Quaternion]] = []
normalForce = []
shearForce = []
bendingMoment = []
torsionMoment = []


def printStuff():
    # [print("normal force: ", inter.normal_force) for _, inter in interCont]
    # [print("Shear force: ", inter.shear_force) for _, inter in interCont]
    # [print("Bending moment: ", inter.bending_moment) for _, inter in interCont]
    # [print("Torsion moment: ", inter.torsion_moment) for _, inter in interCont]
    poses = [body.pos for body in bodyCont.bodies]
    oris = [body.ori for body in bodyCont.bodies]   
    normalForce.append(interCont[0, 1].normal_force)
    shearForce.append(interCont[0, 1].shear_force)
    bendingMoment.append(interCont[0, 1].bending_moment)
    torsionMoment.append(interCont[0, 1].torsion_moment)
    bodyPoses.append(poses)
    bodyOris.append(oris)
    # print("body 1 force", bodyCont[0].force)
    # print("body 2 force", bodyCont[1].force)
    # print(" ")

In [ ]:
simulationLoop = SimulationLoop(
    engines=[
        ForceResetter(),
        InteractionsCalculator(),
        LeapFrogIntegrator(dt=1e-6),
        CustomPythonEngine(pyFunction=printStuff)
    ]
)
simulationLoop.simulate(50_000)

In [ ]:
def plot_2d_scatter(n=0):
    xs = [i[0] for i in bodyPoses[n]]
    ys = [i[1] for i in bodyPoses[n]]
    plt.scatter(xs, ys)
    plt.xlim(-3, 3)
    plt.ylim(0, 6)

interact(plot_2d_scatter, n=(0,50000,1))

In [ ]:
fx = [f[0] for f in shearForce]
fy = [f[1] for f in shearForce]
fz = [f[2] for f in shearForce]

plt.plot(fx, label="x")
plt.plot(fy, label="y")
plt.plot(fz, label="z")
plt.legend()

In [ ]:
lens = [norm(pos1 - pos2) for pos1, pos2 in bodyPoses]
plt.plot(lens)
plt.ylim(0.99, 1.01)

In [ ]:
angles = [ori1.conv_2axisAngle().angle for ori1, ori2 in bodyOris]
axes = [ori1.conv_2axisAngle().axis for ori1, ori2 in bodyOris]

plt.plot(angles)